In [31]:
import os
import re
import pickle
import neurokit2 as nk
import sensors_import_support as sis
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import glob

In [2]:
# Import questionnaires and subjects data
with open('questionnaires.pkl', 'rb') as f:
    subject_data = pickle.load(f)

subject_data

,ID,Task,Number,STAI_Total,Fatigue,Stress,Mental_Demand,Physical_Demand,Frustration,NASA_Total,PSS,NARS_S1,NARS_S2,NARS_S3,Age,Sex,Experience
0,8.0,MANUAL,1,39.0,2.0,8.0,6.0,6.0,1.0,72,16,9,9,9,26.0,1,1.0
1,8.0,MANUAL,2,51.0,2.0,13.0,4.0,5.0,1.0,71,16,9,9,9,26.0,1,1.0
2,8.0,MANUAL,3,42.0,2.0,10.0,4.0,4.0,2.0,72,16,9,9,9,26.0,1,1.0
3,8.0,MANUAL,4,36.0,1.0,9.0,3.0,3.0,1.0,55,16,9,9,9,26.0,1,1.0
4,8.0,MANUAL,5,50.0,2.0,12.0,3.0,2.0,4.0,64,16,9,9,9,26.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,1.0,COBOT,11,59.0,3.0,17.0,3.0,5.0,6.0,63,12,16,14,11,26.0,1,1.0
332,1.0,COBOT,12,61.0,4.0,19.0,3.0,5.0,6.0,63,12,16,14,11,26.0,1,1.0
333,1.0,COBOT,13,61.0,4.0,19.0,3.0,5.0,6.0,63,12,16,14,11,26.0,1,1.0
334,1.0,COBOT,14,61.0,4.0,19.0,3.0,5.0,6.0,63,12,16,14,11,26.0,1,1.0


In [ ]:
# Import signals
signals, fs = sis.import_opensignals('signals_example.txt')
signals[0:10]

In [ ]:
# Plot ecg with R peaks (only a subset, otherwise the plot becomes unreadable!)
ecg_processed, ecg_info = nk.ecg_process(signals['ECG'], sampling_rate=fs)
rpeaks = ecg_info["ECG_R_Peaks"]
cleaned_ecg = ecg_processed["ECG_Clean"]
plot = nk.events_plot(rpeaks[0:13], cleaned_ecg[0:10000])

In [ ]:
# Plot filtered emg
emg_processed, _ = nk.emg_process(signals['EMG'], sampling_rate=fs)
plot = nk.emg_plot(emg_processed, fs)

In [ ]:
# Plot filtered eda
eda_processed, features = nk.eda_process(signals['EDA'], sampling_rate=fs, method="neurokit")
plot = nk.eda_plot(eda_processed, fs)

In [4]:
# Extraction of features

fs = 1000
window = 30
experiment_types = ['manual', 'cobot', 'stroopeasy', 'stroophard', 'rest']

If you did not already, please run the patient divider before the following part of the code to have the right project structure.

In [42]:
def extract_and_save_ECG(load_path, experiment_type):
    # ECG 
    # if file 06-05-30_10-53-13 is present please delete it.
    for directory in os.listdir(load_path):
        path = os.path.join(load_path, directory)
            
        # this splits both slash and dot to take only the filename without extention
        filenames = [re.split(r'[/.]', i)[-2] for i in glob.glob(os.path.join(path, '*.txt'))]
        patient_id = int(directory.split('-')[1])
        
        dataframe_ECG = sis.extract_plux_data_windowed(path, 0, window, 'ECG')
        ecg_time_and_freq_features = sis.extract_time_and_freq_hrv_features(dataframe_ECG, fs, patient_id, filenames)
        ecg_time_and_freq_features.to_csv(Path("saved_data", directory, f"ecg_{experiment_type}_{patient_id}.csv"), index=False)

In [43]:
def extract_and_save_EDA(load_path, experiment_type):
    # EDA
    for directory in os.listdir(load_path):
        path = os.path.join(load_path, directory)
        
        # this splits both slash and dot to take only the filename without extention
        filenames = [re.split(r'[/.]', i)[-2] for i in glob.glob(os.path.join(path, '*.txt'))]
        patient_id = int(directory.split('-')[-1])
        
        dataframe_EDA = sis.extract_plux_data_windowed(path, 0, window, 'EDA')
        eda_time_and_freq_features = sis.extract_eda_time_and_frequency_features(dataframe_EDA, fs, window, patient_id, filename)
        eda_time_and_freq_features.to_csv(Path("saved_data", directory, f"eda_{experiment_type}_{patient_id}.csv"), index=False)

In [44]:
def extract_and_save_EMG(load_path, experiment_type):
    # EMG
    for directory in os.listdir(load_path):
        path = os.path.join(load_path, directory)
        
        # this splits both slash and dot to take only the filename without extention
        filenames = [re.split(r'[/.]', i)[-2] for i in glob.glob(os.path.join(path, '*.txt'))]
        patient_id = int(directory.split('-')[-1])

        dataframe_EMG = sis.extract_plux_data_windowed(path, 0, window, 'EMG')
        emg_time_and_freq_features = sis.extract_emg_features(dataframe_EMG, fs, patient_id, filename)
        emg_time_and_freq_features.to_csv(Path("saved_data", directory, f"emg_{experiment_type}_{patient_id}.csv"), index=False)

In [45]:
    for t in experiment_types:
        load_path = os.path.join(os.getcwd(), 'experiments', t)

        extract_and_save_ECG(load_path, t)
        extract_and_save_EDA(load_path, t)
        extract_and_save_EMG(load_path, t)

['02-05-23_10-26-10_manual', '02-05-23_11-33-37_manual', '02-05-23_11-26-48_manual', '02-05-23_10-18-42_manual', '02-05-23_09-53-36_manual', '02-05-23_11-19-38_manual', '02-05-23_10-10-57_manual', '02-05-23_10-58-35_manual', '02-05-23_11-40-10_manual', '02-05-23_09-45-48_manual', '02-05-23_09-34-14_manual', '02-05-23_11-06-02_manual', '02-05-23_11-12-30_manual', '02-05-23_10-02-49_manual']
['05-06-07_15-02-41_manual', '05-06-07_16-25-36_manual', '05-06-07_16-20-43_manual', '05-06-07_16-11-42_manual', '05-06-07_15-10-22_manual', '05-06-07_15-40-39_manual', '05-06-07_16-31-28_manual', '05-06-07_15-16-52_manual', '05-06-07_16-02-40_manual', '05-06-07_15-28-48_manual', '05-06-07_15-46-03_manual', '05-06-07_15-22-28_manual', '05-06-07_15-34-30_manual', '05-06-07_16-07-12_manual', '05-06-07_16-16-14_manual']
['04-05-23_17-48-53_manual', '04-05-23_18-19-52_manual', '04-05-23_18-13-44_manual', '04-05-23_17-42-45_manual', '04-05-23_17-54-24_manual', '04-05-23_17-14-19_manual', '04-05-23_18-37-3

Now all the patients are divided in their respective folder, and every experiment type is divided too (manual divided from the orther types, and ecg divided from eda and emg).

Then we can decide if we want to merge by patient or other options.